# RAG using Meta AI Llama-3


<img src="./resources/rag_architecture.png" width=800px>

In [1]:
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader

In [2]:
# allows nested access to the event loop
nest_asyncio.apply()

In [3]:
# add your documents in this directory, you can drag & drop
input_dir_path = '/teamspace/studios/this_studio/test-dir'

In [3]:

# setup llm & embedding model
llm=Ollama(model="llama3", request_timeout=120.0)
# embed_model = HuggingFaceEmbedding( model_name="Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)

embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

In [4]:
import os
os.environ['ACTIVELOOP_TOKEN'] = "eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpZCI6Imh1bnRlciIsImFwaV9rZXkiOiJ6ejh5dG1sZURjNWkwc3g3NldNRjF5NWNrUWNIMnlfenR2VHV6MERmM3ppMVAifQ."

In [5]:
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

my_activeloop_org_id = "hunter"
my_activeloop_dataset_name = "law-vn-langchain-advanced-rag"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

Deep Lake Dataset in hub://hunter/law-vn-langchain-advanced-rag already exists, loading from the storage


In [8]:
# load data
# loader = SimpleDirectoryReader(
#             input_dir = input_dir_path,
#             required_exts=[".pdf"],
#             recursive=True
#         )
# docs = loader.load_data()

# Creating an index over loaded data
Settings.embed_model = embed_model

from llama_index.core.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, show_progress=True)

# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine()

# ====== Customise prompt template ======
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response
response = query_engine.query("Hối lộ hơn 20 nghìn tỷ thì sẽ bị gì",)

BG QUEUE ERROR: tensor::array_for_range works only on static tensors. Static tensors have numeric dtype, null compression, are not links and are not sequences.


In [6]:
Settings.embed_model = embed_model

from llama_index.core.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, show_progress=True)

# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine()

In [7]:
# ====== Customise prompt template ======
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Dựa vào context trên, hãy nghĩ theo thứ tự từng bước để trả lời câu hỏi 1 cách chính xác,súc tích. Trường hợp bạn không biết thì trả lời 'Tôi đéo biết'.\n"
"Query: {query_str}\n"
"Answer: "
)

In [8]:
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response
response = query_engine.query("Hối lộ hơn 20 nghìn tỷ thì sẽ bị gì",)

BG QUEUE ERROR: tensor::array_for_range works only on static tensors. Static tensors have numeric dtype, null compression, are not links and are not sequences.


In [ ]:
display(Markdown(str(response)))

Based on the provided context, DSPy (Declarative Programming for Self-Refining Language Model Pipelines) is a framework for programmatically solving advanced tasks with language and retrieval models through composing and declaring modules. It aims to replace brittle "prompt engineering" tricks with composable modules and automatic (typically discrete) optimizers.

In other words, DSPy provides a way to define signatures that specify what an LM needs to do declaratively, allowing developers to create modular functions that support any signature. This enables the creation of complex language pipelines that can be composed in arbitrary ways and optimized for specific tasks.